# Climate Risk Analysis: Mongolia Dzud and Summer Drought Analysis 2009-2010

## Introduction

**Objectives**

## Data Sources

### Primary Sources

### Supplementary Sources

### Data Notes

## Historical Baseline Context

### Why 1981-2009?

### Statistical References

## Summer 2009 Preconditioning Analysis

## Winter 2009-2010 Dzud Severity Analysis

## Localised Summer and Winter Analysis: Ulaanbaatar

## Seasonal Patterns and Compounding Risks

## Summary Statistics

## Conclusion and Next Steps